<a href="https://colab.research.google.com/github/hopebrooke/3931/blob/main/updatedDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import DeePore as dp
import numpy as np
import h5py

In [ ]:
# dp.show_feature_maps doesn't work on ipynb due to floats/int conversion
# redefining here with same code but added casting
def show_feature_maps(A):
  N= int(np.ceil(np.sqrt(A.shape[0])))
    f=plt.figure(figsize=(N*10,N*10))
    for I in range(A.shape[0]):
        plt.subplot(N,N,I+1)
        plt.imshow(dp.normal(np.squeeze(A[I,:,:,:])))
        plt.axis('off')
    plt.show()

    f.savefig('images/initial_feature_maps.png')

# using monkey patching to update orginal function (not usually advised!)
dp.show_feature_maps = show_feature_maps

In [ ]:
# redefining functions for changing slice volume
# same as dp.slicevol but getting a different number of slices at different intervals

# function that takes 2 slices from each direction
def slicevol_2(A):
    A=np.squeeze(A)
    B=np.zeros((1,A.shape[0],A.shape[1],6))
    # slices from front
    B[0,:,:,0]=A[int(A.shape[0]/3),:,:]
    B[0,:,:,1]=A[int(2 * A.shape[0]/3),:,:]
    # slices from left
    B[0,:,:,2]=A[:,int(A.shape[1]/3),:]
    B[0,:,:,3]=A[:,int(2 * A.shape[1]/3),:]
    # slices from top
    B[0,:,:,4]=A[:,:,int(A.shape[2]/3)]
    B[0,:,:,5]=A[:,:,int( 2 * A.shape[2]/3)]
    return B

# function that takes 3 slices from each direction
def slicevol_3(A)
    A=np.squeeze(A)
    B=np.zeros((1,A.shape[0],A.shape[1],9))
    # slices from front
    B[0,:,:,0]=A[int(A.shape[0]/4),:,:]
    B[0,:,:,1]=A[int(2 * A.shape[0]/4),:,:]
    B[0,:,:,2]=A[int(3 * A.shape[0]/4),:,:]
    # slices from left
    B[0,:,:,3]=A[:,int(A.shape[1]/4),:]
    B[0,:,:,4]=A[:,int(2 * A.shape[1]/4),:]
    B[0,:,:,5]=A[:,int(3 * A.shape[1]/4),:]
    # slices from top
    B[0,:,:,6]=A[:,:,int(A.shape[2]/4)]
    B[0,:,:,7]=A[:,:,int( 2 * A.shape[2]/4)]
    B[0,:,:,8]=A[:,:,int( 3 * A.shape[2]/4)]
    return B

In [ ]:
# if we change slice volume, we change how the compact dataset is created...
# same as dp.create_compact_dataset but changing parameters for writeh5slice() call

# for 2 slice in each direction:
def create_compact_dataset_2(Path_complete,Path_compact):
    S=hdf_shapes(Path_complete,['X'])
    for I in range(S[0][0]):
        X=readh5slice(Path_complete,'X',[I])
        Y=readh5slice(Path_complete,'Y',[I])
        X=slicevol(X)
        X=ecl_distance(X)
        writeh5slice(X,Path_compact,'X',Shape=[128,128,6])
        writeh5slice(Y,Path_compact,'Y',Shape=[1515,1])

# for 3 slice in each direction
def create_compact_dataset_3(Path_complete,Path_compact):
    S=hdf_shapes(Path_complete,['X'])
    for I in range(S[0][0]):
        X=readh5slice(Path_complete,'X',[I])
        Y=readh5slice(Path_complete,'Y',[I])
        X=slicevol(X)
        X=ecl_distance(X)
        writeh5slice(X,Path_compact,'X',Shape=[128,128,9])
        writeh5slice(Y,Path_compact,'Y',Shape=[1515,1])